In [1]:
# data feeds
import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import math
import pandas as pd

In [2]:
# 從Yahoo Finance取得資料
# data = btfeeds.YahooFinanceData(dataname='SPY', 
#                                 fromdate=datetime.datetime(2019, 1, 1),
#                                 todate=datetime.datetime(2019, 12, 31))

In [3]:
class PandasData(bt.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
    )
    # datafields = PandasData.datafields + (['SRNE'])

In [4]:
dataframe = pd.read_csv('test.csv', delimiter=",", index_col="datetime", parse_dates= True)
data = bt.feeds.PandasData(dataname=dataframe)

In [5]:
dataframe.head()

,close,open,high,low,volume,openinterest
datetime,,,,,,
2010-10-19,26.35,27.0,27.05,26.30,3659159.0,0.584795
2010-10-20,26.10,26.4,26.65,26.00,2343239.0,0.625937
2010-10-21,26.30,26.0,26.50,25.90,2505356.0,0.395891
2010-10-22,27.10,26.3,27.20,26.25,5585503.0,0.425043
2010-10-25,26.80,27.1,27.40,26.80,4753909.0,0.470111


In [8]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.openinterest = self.datas[0].openinterest

    def next(self):
         # 帳戶沒有部位
        if not self.position:
            # 5ma往上穿越20ma
            if self.openinterest[0] > 0.5:
                # 印出買賣日期與價位
                self.log('BUY ' + ', Price: ' + str(self.dataopen[0]))
                # 使用開盤價買入標的
                self.buy(price=self.dataopen[0])
        # 5ma往下穿越20ma
        elif self.openinterest[0] < 0.4:
            # 印出買賣日期與價位
            self.log('SELL ' + ', Price: ' + str(self.dataopen[0]))
            # 使用開盤價賣出標的
            self.close(price=self.dataopen[0])

In [9]:
# 初始化cerebro
cerebro = bt.Cerebro()
# feed data
cerebro.adddata(data)
# add strategy
cerebro.addstrategy(TestStrategy)
# run backtest
cerebro.run()
# plot diagram
cerebro.plot()

2010-10-19, BUY , Price: 27.0
2010-10-21, SELL , Price: 26.0
2010-10-26, BUY , Price: 27.1
2010-11-24, SELL , Price: 26.7
2010-11-25, BUY , Price: 26.3
2010-12-03, SELL , Price: 27.6
2010-12-13, BUY , Price: 27.7
2010-12-17, SELL , Price: 27.65
2010-12-24, BUY , Price: 28.2
2011-01-04, SELL , Price: 29.0
2011-01-10, BUY , Price: 28.8
2011-04-22, SELL , Price: 24.4
2011-05-03, BUY , Price: 25.95
2011-05-10, SELL , Price: 25.5
2011-05-16, BUY , Price: 25.8
2011-05-30, SELL , Price: 23.95
2011-06-13, BUY , Price: 24.95
2011-06-23, SELL , Price: 24.2
2011-06-24, BUY , Price: 24.1
2011-07-11, SELL , Price: 24.6
2011-07-13, BUY , Price: 24.1
2011-07-15, SELL , Price: 23.7
2011-07-19, BUY , Price: 24.4
2011-07-22, SELL , Price: 24.4
2011-07-26, BUY , Price: 24.85
2011-07-28, SELL , Price: 25.2
2011-08-01, BUY , Price: 25.35
2011-08-16, SELL , Price: 20.05
2011-08-18, BUY , Price: 20.25
2011-09-19, SELL , Price: 19.05
2011-09-21, BUY , Price: 19.1
2011-10-11, SELL , Price: 17.2
2011-10-17, BUY

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]